<a href="https://colab.research.google.com/github/jabb4/AlexNet_Pokedex/blob/main/myPokedex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Pokemon Images classification using AlexNet-based CNN architecture

 
 

In [1]:
#!pip3 install opendatasets
#!pip3 install import_ipynb
!pip3 install kaggle
import import_ipynb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import time 

from os import listdir
import os
import cv2 as cv
import glob
import opendatasets as od

from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T
import torch
import torch.nn as nn
from torch.optim import lr_scheduler 
from torchvision import transforms, utils
from torchvision.utils import make_grid
from torchvision.utils import save_image
from torchvision.io import read_image
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split

from skimage import io, transform
import torch.optim as optim

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
/home/jack/.local/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# model = AlexNet.AlexNet()
import  sys
sys.path.append('/content/drive/MyDrive/Colab_Notebooks/project_exam/AlexNet_Pokedex/')
from tools.AlexNet import AlexNet
from tools.images_tools import *

importing Jupyter notebook from /home/jack/myPokedex/AlexNet_Pokedex/tools/AlexNet.ipynb
importing Jupyter notebook from /home/jack/myPokedex/AlexNet_Pokedex/tools/images_tools.ipynb


In [3]:
od.download("https://www.kaggle.com/datasets/thedagger/pokemon-generation-one")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: 

In [ ]:
root_path = '/content/drive/MyDrive/Colab_Notebooks/project_exam/AlexNet_Pokedex/pokemon-generation-one/dataset/'
classes = os.listdir(root_path)
count=0
count_dict = {}
print(f'Total number of pokemons: {len(classes)}')
for pokemon in classes:
    dir_path = os.path.join(root_path, pokemon)
    count+=len(os.listdir(dir_path))
    count_dict[pokemon] = len(os.listdir(dir_path))

In [ ]:
pd.Series(count_dict).sort_values(ascending=False).plot.bar(figsize=(30,10))

In [ ]:
from torch.nn.common_types import Tensor
class PokemonImageDataset(Dataset):
  """ Pokemon images dataset """
  
  def __init__(self, list_of_pokemon_images, images_labels, transform = None):
    """
    Args:
        list_of_pokemon_images    list[string]: list of pokemon file path
        images_labels             list[string]: list of pokemon labels for each image
        transform                 (callable, optional): Optional transform to be applied on a semple
    """
    self.list_of_pokemon_images = list_of_pokemon_images
    self.images_labels          = images_labels
    self.transform              = transform

  def __len__(self):
    " returns the number of samples (images) in our dataset. "
    return len(self.list_of_pokemon_images)
  
  def __getitem__(self, idx : int) -> dict:
    """
    The __getitem__ function loads and returns a sample from the dataset at the given index idx. 
    Based on the index, it identifies the image’s location on disk, converts that to a tensor using read_image, 
    retrieves the corresponding label, 
    calls the transform functions on them (if applicable), and returns the tensor image and corresponding label in a dictionary.
    """
    if torch.is_tensor(idx): idx = idx.tolist()
    
    try: 
      image = read_image(self.list_of_pokemon_images[idx])
      label = self.images_labels[idx]
    except:
      image = read_image(self.list_of_pokemon_images[0])
      # print(f"image type {image.dtype}")
      label = self.images_labels[0]

    nof_channels = image.shape[0]

    if(nof_channels == 4): #deal with 4 channels -> cut 1 channel
      image = image[:3,:]
    elif(nof_channels == 1): #deal with 1  channels -> extend to 3 channels (2 redundant)
      image = image.expand(3,*image.shape[1:])


    sample = {"image":image, "label":label}
    
    if self.transform:
      sample = self.transform(sample)
    return sample




In [ ]:
root_path = '/content/drive/MyDrive/Colab_Notebooks/project_exam/AlexNet_Pokedex/pokemon-generation-one/dataset/'

list_of_pokemons=['Mewtwo', 'Pikachu', 'Charmander', 'Bulbasaur', 'Squirtle']

numerical_label = dict(zip(list_of_pokemons,range(len(list_of_pokemons))))

list_of_images = [file for pokemon in list_of_pokemons for file in glob.glob(root_path + pokemon + "/*")]

list_of_labels = []

for image_path in list_of_images:
  for name, label in numerical_label.items():
    if name in image_path: list_of_labels.append(numerical_label[name])

In [ ]:
print(f"example: image -> {list_of_images[0]}, label -> {list_of_labels[0]}")
print(f"example: image -> {list_of_images[1000]}, label -> {list_of_labels[1000]}")
print(f"number of images : {len(list_of_images)}, number of labels : {len(list_of_labels)}")

In [ ]:
pokemon_dataset = PokemonImageDataset(list_of_images, list_of_labels)

In [ ]:
fig,ax = plt.subplots(1,4,figsize=(12,8))

for i, sample in enumerate(pokemon_dataset):
  print("sample : ", i, sample['image'].shape)
  plt.tight_layout()
  ax[i].set_title('Sample #{}'.format(i))
  # ax[i].axis('off')
  
  try:
    ax[i].imshow(sample["image"])
  except:
    ax[i].imshow(sample["image"][0])
    # print("4 images resized to 3 k")
  if i==3: break

In [ ]:
scale = Rescale(256)
crop = RandomCrop(128)
composed = transforms.Compose([Rescale(256), RandomCrop(224)])

In [ ]:
fig,ax = plt.subplots(1,4,figsize=(20,10))

sample = pokemon_dataset[38] # what if image has 4k resolution ?

ax[0].imshow(sample["image"].transpose(0,1).transpose(1,2))
ax[0].set_title("Original")

for i, tsfrm in enumerate([scale, crop, composed]):  
  transformed_sample = tsfrm(sample)
  # if(i==2):
  #     mean = (transformed_sample["image"].float()[0].mean(), transformed_sample["image"].float()[1].mean(), transformed_sample["image"].float()[2].mean())
  #     std  = (transformed_sample["image"].float()[0].std(),  transformed_sample["image"].float()[1].std(),  transformed_sample["image"].float()[2].std())
  #     norm = Normalize(mean, std)
  #     transformed_sample = norm(transformed_sample)

  ax[i+1].set_title(type(tsfrm).__name__)
  ax[i+1].imshow(transformed_sample["image"].transpose(0,1).transpose(1,2))

## **DATALOADERS**

In [ ]:
pokemon_dataset = PokemonImageDataset(list_of_pokemon_images = list_of_images, 
                                      images_labels          = list_of_labels,
                                      transform              = transforms.Compose([Rescale(256),
                                                                                   RandomCrop(224)]
                                                                                  ))

pokemon_dataloader = DataLoader(pokemon_dataset, batch_size = 4, shuffle = True, num_workers=0)

In [ ]:
for i_batch, sample_batched in enumerate(pokemon_dataloader):
  image = sample_batched["image"]
  label = sample_batched["label"]
  print(f"batch number : {i_batch}, batch shape : {image.size()}, batch[0] shape{image[0].size()}, label {label}")
  
  if i_batch == 3:
    images_batch, label_batch = sample_batched['image'],sample_batched['label']
    batch_len = images_batch.shape[0]
    fig,ax = plt.subplots(1,batch_len, figsize = (20,10))
    for i in range(batch_len):
      ax[i].imshow(images_batch[i].permute(1,2,0))
      ax[i].set_title(f"true label : {list_of_pokemons[label_batch[i]]}")
    break

In [ ]:
train, test_set = train_test_split(pokemon_dataset, test_size=0.2, train_size = 0.8, random_state = 42,shuffle=True)

In [ ]:
train_set, validation_set =  train_test_split(train, test_size=0.25, train_size = 0.75, random_state = 42,shuffle=True)

In [ ]:
print("training set size : {}, validation set size : {}, test set size : {}".format(len(train_set),len(validation_set),len(test_set)))

In [ ]:
dataloaders = {'train': DataLoader(train_set, batch_size = 10, shuffle = True, num_workers=0), 
               'val'  : DataLoader(validation_set, batch_size = 10, shuffle = True, num_workers=0), 
               'test' : DataLoader(test_set, batch_size = 10, shuffle = True, num_workers=0)} 

dataset_sizes = {x: y for x,y in zip(['train', 'val','test'],[len(train_set),len(validation_set),len(test_set)])}

In [ ]:
print(dataset_sizes["val"])

In [ ]:
for dictionary in dataloaders['train']:
  print(dictionary['image'].shape)
  if i == 3: break

## TRAINING THE NETWORK


In [ ]:
model = AlexNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
num_epochs = 5



In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

## SPLIT : TRAINING VALIDATION & TEST SET

In [ ]:
since = time.time()
best_accuracy = 0.

train_accuracy = []
validation_accuracy = []

train_loss = []
validation_loss = []

for epoch in range(num_epochs):
  print(f'Epoch {epoch}/{num_epochs - 1}')
  print('-'*10)

  for phase in ['train','val']:
    if phase == 'train':
      model.train() # set model to training mode
    else:
      model.eval()   # set model to evaluate mode (tested on the validation set first)

    running_loss, running_corrects = 0.0, 0

    for running_data in dataloaders[phase]:
      images = running_data['image']
      labels = running_data['label']

      # print(f"images shape : {images.shape} , labels : {labels}")

      # zero the parameter gradients
      optimizer.zero_grad()
      # print("setting zero gradients")

      # forward
      # track history if only in train

      with torch.set_grad_enabled(phase == 'train'):
        # print("Alla fine l'input gli è piaciuto")
        outputs = model(images.float())
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

      # backward + optimize only if in training phase
        if phase == 'train':
          loss.backward()
          optimizer.step()

      running_loss += loss.item() * images.size(0)
      running_corrects += torch.sum(preds == labels.data)
    # (Da rivedere) call of the progress step per epoch
    if phase == 'train': 
      scheduler.step()
    
    #loss and accuracy evaluation per each epoch
    epoch_loss = running_loss / dataset_sizes[phase]
    epoch_accuracy = running_corrects.double() / dataset_sizes[phase]

    print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_accuracy:.4f}')
    if phase == 'train':
      train_accuracy.append(epoch_accuracy)
      train_loss.append(epoch_loss)
    if phase == 'val':
      validation_accuracy.append(epoch_accuracy)
      validation_loss.append(epoch_loss)
      
    if phase == 'val' and epoch_accuracy > best_accuracy:
      best_accuracy = epoch_accuracy
      # best_model_wts = copy.deepcopy(model.state_dict())
    
    print()

time_elapsed = time.time() - since
print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
print(f'Best val Acc: {best_accuracy:4f}')

# load best model weights
# model.load_state_dict(best_model_wts)





In [ ]:
model.eval()
was_training = model.training

with torch.no_grad():
  for running_data in dataloaders['test']:
    images = running_data['image']
    labels = running_data['label']

    outputs = model(images.float())
    _, preds = torch.max(outputs, 1)

    fig,ax = plt.subplots(1,10, figsize=(30,15))
    for index,images in enumerate(images):
      ax[index].imshow(images.permute(1,2,0))
      ax[index].set_title(f"predicted: {list_of_pokemons[preds[index]]}")

    break

# Plots

In [ ]:
plt.plot(train_accuracy,'-o')
plt.plot(validation_accuracy,'-o')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train','Validation'])
plt.title('Train vs Validation Accuracy')
plt.show()


In [ ]:
plt.plot(train_loss,'-o')
plt.plot(validation_loss,'-o')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train','Validation'])
plt.title('Train vs Validation Loss')
plt.show()


In [ ]:
pokemon_dataset[1]['image'].shape

In [ ]:
test = pokemon_dataset[3]["image"]
test.shape

In [ ]:
plt.imshow(test.permute(1,2,0))

In [ ]:
plt.imshow(pokemon_dataset[10]['image'][:3,:].permute(1,2,0)) # deal with 4 channel images -> eliminate 1 channel

In [ ]:
plt.imshow(pokemon_dataset[1]['image'].expand(3,*pokemon_dataset[1]['image'].shape[1:]).permute(1,2,0)) # deal with 1 channel images